In [1]:
import sys
print(sys.version)

3.11.10 | packaged by Anaconda, Inc. | (main, Oct  3 2024, 07:22:26) [MSC v.1929 64 bit (AMD64)]


In [2]:
import dask.dataframe as dd


#import dask
#print(dask.__version__)


In [3]:
!pip install dask[dataframe]


In [4]:
#import dask.dataframe as dd
import time
#from dask.distributed import Client
#client = Client()  

In [5]:
import pandas as pd

# Create a large pandas DataFrame and save it as a CSV file
df = pd.DataFrame({
    'A': range(1, 1000001),  # 1 million rows
    'B': range(1000000, 0, -1),
    'C': range(500000, 1500000)
})

df1=pd.DataFrame({
    'A': range(1, 1000001),  # 1 million rows
    'B': range(1000000, 0, -1),
    'C': range(500000, 1500000)
})
#df.to_csv('large_dataset.csv', index=False)


In [6]:
#import dask.dataframe as dd
#df = dd.read_csv('large_dataset.csv')
#Each partition represents a piece of the data.
dff = dd.from_pandas(df,npartitions=3)
#computes the DataFrame and stores it in memory (RAM),optimize performance.
dff = dff.persist()
# Preview the data
print(dff.head())
print("")
print(dff.tail())
print("")
print(dff.npartitions)
print(dff.dtypes)
print(dff.info())

   A        B       C
0  1  1000000  500000
1  2   999999  500001
2  3   999998  500002
3  4   999997  500003
4  5   999996  500004

              A  B        C
999995   999996  5  1499995
999996   999997  4  1499996
999997   999998  3  1499997
999998   999999  2  1499998
999999  1000000  1  1499999

3
A    int64
B    int64
C    int64
dtype: object
<class 'dask_expr.DataFrame'>
Columns: 3 entries, A to C
dtypes: int64(3)None


In [7]:
shape = dff.shape  # Returns a Dask array representing the shape

# Compute the number of rows
num_rows = dff.shape[0].compute()  # Number of rows

# Get the number of columns directly
num_columns = dff.shape[1]  # Number of columns (no compute needed)

# Get the columns of the Dask DataFrame
columns_result = dff.columns.tolist()  # Convert columns to list

print("Number of Rows:", num_rows)
print("Number of Columns:", num_columns)
print("Columns of Dask DataFrame:", columns_result)

Number of Rows: 1000000
Number of Columns: 3
Columns of Dask DataFrame: ['A', 'B', 'C']


# loc,iloc

In [8]:
#loc[]: Selects rows by label and includes the endpoint.
#iloc[]: Selects rows by integer position and excludes the endpoint. 
print(dff.loc[0:100].compute())   

#selected_rows_iloc = dff.iloc[0:100, :].compute()
#print(selected_rows_iloc)

#Only indexing the column positions is supported. Trying to select row positions will raise a ValueError.
ss=df.iloc[:, [2, 0, 1]]
print(ss)

       A        B       C
0      1  1000000  500000
1      2   999999  500001
2      3   999998  500002
3      4   999997  500003
4      5   999996  500004
..   ...      ...     ...
96    97   999904  500096
97    98   999903  500097
98    99   999902  500098
99   100   999901  500099
100  101   999900  500100

[101 rows x 3 columns]
              C        A        B
0        500000        1  1000000
1        500001        2   999999
2        500002        3   999998
3        500003        4   999997
4        500004        5   999996
...         ...      ...      ...
999995  1499995   999996        5
999996  1499996   999997        4
999997  1499997   999998        3
999998  1499998   999999        2
999999  1499999  1000000        1

[1000000 rows x 3 columns]


In [9]:
#Without .compute(), Dask only builds a task graph and won't show the actual data.
#access a particular partition
print(dff.partitions[0].compute())

             A        B       C
0            1  1000000  500000
1            2   999999  500001
2            3   999998  500002
3            4   999997  500003
4            5   999996  500004
...        ...      ...     ...
333329  333330   666671  833329
333330  333331   666670  833330
333331  333332   666669  833331
333332  333333   666668  833332
333333  333334   666667  833333

[333334 rows x 3 columns]


# select columns

In [10]:
# Select a column
print(dff['A'].compute())
print(dff[["C","B"]].compute())

# Filter rows where 'A' is greater than 500,000
filtered_df = dff[dff['A'] > 500000].compute()
print(filtered_df.head())


0               1
1               2
2               3
3               4
4               5
           ...   
999995     999996
999996     999997
999997     999998
999998     999999
999999    1000000
Name: A, Length: 1000000, dtype: int64
              C        B
0        500000  1000000
1        500001   999999
2        500002   999998
3        500003   999997
4        500004   999996
...         ...      ...
999995  1499995        5
999996  1499996        4
999997  1499997        3
999998  1499998        2
999999  1499999        1

[1000000 rows x 2 columns]
             A       B        C
500000  500001  500000  1000000
500001  500002  499999  1000001
500002  500003  499998  1000002
500003  500004  499997  1000003
500004  500005  499996  1000004


# groupby & aggregation

In [11]:
#same as in pandas
print(dff.groupby("A")["C"].mean().compute()) #same values of c
print("")
#GroupBy with Multiple Aggregations
print(dff.groupby("A")['B'].agg(['sum', 'mean', 'min', 'max']).compute()) #same values of b


A
1           500000.0
2           500001.0
3           500002.0
4           500003.0
5           500004.0
             ...    
999996     1499995.0
999997     1499996.0
999998     1499997.0
999999     1499998.0
1000000    1499999.0
Name: C, Length: 1000000, dtype: float64

             sum       mean      min      max
A                                            
1        1000000  1000000.0  1000000  1000000
2         999999   999999.0   999999   999999
3         999998   999998.0   999998   999998
4         999997   999997.0   999997   999997
5         999996   999996.0   999996   999996
...          ...        ...      ...      ...
999996         5        5.0        5        5
999997         4        4.0        4        4
999998         3        3.0        3        3
999999         2        2.0        2        2
1000000        1        1.0        1        1

[1000000 rows x 4 columns]


# create new column and apply sqrt()

In [13]:
#use apply to create new column or apply function to each row or column elemnet
import math
dff['A_sqrt'] = dff['A'].apply(math.sqrt, meta=('A_sqrt', 'f8'))
result=dff.compute()
print(result)

              A        B        C       A_sqrt
0             1  1000000   500000     1.000000
1             2   999999   500001     1.414214
2             3   999998   500002     1.732051
3             4   999997   500003     2.000000
4             5   999996   500004     2.236068
...         ...      ...      ...          ...
999995   999996        5  1499995   999.998000
999996   999997        4  1499996   999.998500
999997   999998        3  1499997   999.999000
999998   999999        2  1499998   999.999500
999999  1000000        1  1499999  1000.000000

[1000000 rows x 4 columns]


# Statistical Operations:

In [14]:
#tha same in pandas
print(dff["A"].mean().compute())
print(dff["A"].min().compute())
print(dff["A"].max().compute())
print(dff["A"].sum().compute())

500000.5
1
1000000
500000500000



# concate dask along rows& columns

In [15]:
dff1= dd.from_pandas(df1,npartitions=3)
concat_dff_rows = dd.concat([dff, dff1]) #along rows(vertically)
print("\nConcatenated Dask DataFrame (rows):\n", concat_dff_rows.compute())

# Concatenate Dask DataFrames along columns
concat_ddf_columns = dd.concat([dff, dff1], axis=1)
print("\nConcatenated Dask DataFrame (columns):\n", concat_ddf_columns.compute())



Concatenated Dask DataFrame (rows):
               A        B        C    A_sqrt
0             1  1000000   500000  1.000000
1             2   999999   500001  1.414214
2             3   999998   500002  1.732051
3             4   999997   500003  2.000000
4             5   999996   500004  2.236068
...         ...      ...      ...       ...
999995   999996        5  1499995       NaN
999996   999997        4  1499996       NaN
999997   999998        3  1499997       NaN
999998   999999        2  1499998       NaN
999999  1000000        1  1499999       NaN

[2000000 rows x 4 columns]

Concatenated Dask DataFrame (columns):
               A        B        C       A_sqrt        A        B        C
0             1  1000000   500000     1.000000        1  1000000   500000
1             2   999999   500001     1.414214        2   999999   500001
2             3   999998   500002     1.732051        3   999998   500002
3             4   999997   500003     2.000000        4   999997   50

In [17]:
import pandas as pd
df2 = pd.DataFrame({
    'k': range(0, 50),  
    'l': range(50, 100)  
})
dff2 = dd.from_pandas(df2, npartitions=2)

#The first two rows have NaN for the rolling mean because there are not enough data points to fill the window size of 3.
dff2["rolling_mean"]=dff2["k"].rolling(window=3).mean()
output=dff2.compute()
print(output.head(10))
#print(dff2)



   k   l  rolling_mean
0  0  50           NaN
1  1  51           NaN
2  2  52           1.0
3  3  53           2.0
4  4  54           3.0
5  5  55           4.0
6  6  56           5.0
7  7  57           6.0
8  8  58           7.0
9  9  59           8.0


# set & reset index

In [18]:
#set index # reset
setindex_by_L=dff2.set_index("l").compute()
print(setindex_by_L.head(10))
undo_index=dff2.reset_index().compute()
print(undo_index.head(10))

    k  rolling_mean
l                  
50  0           NaN
51  1           NaN
52  2           1.0
53  3           2.0
54  4           3.0
55  5           4.0
56  6           5.0
57  7           6.0
58  8           7.0
59  9           8.0
   index  k   l  rolling_mean
0      0  0  50           NaN
1      1  1  51           NaN
2      2  2  52           1.0
3      3  3  53           2.0
4      4  4  54           3.0
5      5  5  55           4.0
6      6  6  56           5.0
7      7  7  57           6.0
8      8  8  58           7.0
9      9  9  59           8.0


# Rolling,Cumulative sum dask

In [20]:
#Calculate the rolling sum of column 'k' with a window of 3
dff2['rolling_sum_dask'] = dff2['k'].rolling(window=3).sum()

#Cumulative sum (Dask),dask don't support expanding
dff2['cumsum_dask'] = dff2['k'].cumsum()

# Apply expanding sum
#dff2['A_expanding_sum'] = dff2['k'].expanding().sum()

result_dask = dff2.compute()
print("\nDask DataFrame:")
print(result_dask.head(10))


Dask DataFrame:
   k   l  rolling_mean  rolling_sum_dask  cumsum_dask
0  0  50           NaN               NaN            0
1  1  51           NaN               NaN            1
2  2  52           1.0               3.0            3
3  3  53           2.0               6.0            6
4  4  54           3.0               9.0           10
5  5  55           4.0              12.0           15
6  6  56           5.0              15.0           21
7  7  57           6.0              18.0           28
8  8  58           7.0              21.0           36
9  9  59           8.0              24.0           45


# Rolling,Cumulative sum,expanding pandas

In [21]:
#Rolling sum with a window size of 3
df2['rolling_sum_pandas'] = df2['k'].rolling(window=3).sum() #rolling in pandas & dask give the same results

#Expanding sum
df2['expanding_sum_pandas'] = df2['k'].expanding().sum() #expanding & cumsum give the same results

#Cumulative sum
df2['cumsum_pandas'] = df2['k'].cumsum()

print("Pandas DataFrame:")
print(df2.head(10))

Pandas DataFrame:
   k   l  rolling_sum_pandas  expanding_sum_pandas  cumsum_pandas
0  0  50                 NaN                   0.0              0
1  1  51                 NaN                   1.0              1
2  2  52                 3.0                   3.0              3
3  3  53                 6.0                   6.0              6
4  4  54                 9.0                  10.0             10
5  5  55                12.0                  15.0             15
6  6  56                15.0                  21.0             21
7  7  57                18.0                  28.0             28
8  8  58                21.0                  36.0             36
9  9  59                24.0                  45.0             45


In [23]:
#Convert Data Types:
dff2['new'] = dff2['l'].astype('float64')
output=dff2.compute()
print(output.head(10))

   k   l  rolling_mean  rolling_sum_dask  cumsum_dask   new
0  0  50           NaN               NaN            0  50.0
1  1  51           NaN               NaN            1  51.0
2  2  52           1.0               3.0            3  52.0
3  3  53           2.0               6.0            6  53.0
4  4  54           3.0               9.0           10  54.0
5  5  55           4.0              12.0           15  55.0
6  6  56           5.0              15.0           21  56.0
7  7  57           6.0              18.0           28  57.0
8  8  58           7.0              21.0           36  58.0
9  9  59           8.0              24.0           45  59.0


In [23]:
df3= pd.DataFrame({
    'hello_column': ['Hello World', 'Dask is great', 'DataFrame operations', 'Substring Example'],
     'name_column':['rowida','taher','fcis','IS']
})

# Convert it to a Dask DataFrame with 2 partitions
dff3 = dd.from_pandas(df3, npartitions=2)

dff3['lower_column'] = dff3['hello_column'].str.lower()  # Convert to lowercase

dff3['substring_column'] = dff3['name_column'].str.contains('IS',case=False)# Check for substring case=True by default (case sensitive)
result=dff3.compute()
print(result)

           hello_column name_column          lower_column  substring_column
0           Hello World      rowida           hello world             False
1         Dask is great       taher         dask is great             False
2  DataFrame operations        fcis  dataframe operations              True
3     Substring Example          IS     substring example              True


# query() method is functionally the same in both Dask and pandas

In [24]:
df4 = pd.DataFrame({
    'column1': range(1, 21),  # Values from 1 to 20
    'column2': range(10, 30)   # Values from 10 to 29
})
dff4 = dd.from_pandas(df4, npartitions=2)
#query method is more readible then boolean indexing to filter rows based on string expression
result = dff4.query('column1 > 10 and column2 < 25')
print("Dask DataFrame:")
print(result.compute())

pandas_rs=df4.query('column1 > 10 and column2 < 25')
print("Pandas DataFrame:")
print(pandas_rs)

Dask DataFrame:
    column1  column2
10       11       20
11       12       21
12       13       22
13       14       23
14       15       24
Pandas DataFrame:
    column1  column2
10       11       20
11       12       21
12       13       22
13       14       23
14       15       24


# pivot table to ggregate and reshape data in a DataFrame by pivoting around one or more columns.
The columns attribute in the pivot_table method must be a categorical column. If you try to use a column that is not categorical, you will encounter a ValueError.


In [25]:
#pivot table Often used for summary reports (check sles performance)
#Groupby: More flexible for data manipulation
df5 = pd.DataFrame({
    'column1': ['A', 'A', 'B', 'B', 'C', 'C', 'A', 'B'],
    'column2': ['X', 'Y', 'X', 'Y', 'X', 'Y', 'X', 'Y'],
    'column3': [10, 20, 30, 40, 50, 60, 70, 80]
})
dff5 = dd.from_pandas(df5, npartitions=2)

# Convert 'column2' to categorical it's mandatory
dff5['column2'] = dff5['column2'].astype('category').compute()
#Create a pivot table using Dask
pivot_table = dff5.pivot_table(index='column1', columns='column2', values='column3', aggfunc='mean')

# Compute the result to get the output
pivot_table_result = pivot_table.compute()

# Print the result
print(pivot_table_result)
#rs=dff5.memory_usage(deep=True).compute()  # Check memory usage of each column



column2     X     Y
column1            
A        40.0  20.0
B        30.0  60.0
C        50.0  60.0


# Parallel Computing with Dask

In [27]:
#it's a simple task so dask introduces overhead and takes much time than pandas
#from dask.distributed import Client
from dask import delayed  # Importing delayed function directly
#import time

# Create a Dask client
client = Client()

# The @delayed decorator is used in Dask to turn a normal Python function into a lazy function.
@delayed
def inc(x):
    return x + 1

@delayed
def add(x, y):
    return x + y

start_time = time.time()

a = inc(4).compute()       
b = inc(2)       
c = add(a, b)    

c = c.compute()  
end_time = time.time()

print(c)
print(f"Execution Time with Dask Client: {end_time - start_time} seconds")

NameError: name 'Client' is not defined

In [28]:
#pandas time
def inc(x):
    return x + 1

def add(x, y):
    return x + y

start_time_pandas = time.time()

a = inc(4)  
b = inc(2) 
c = add(a, b)

end_time_pandas = time.time()

print(c) 
print(f"Execution Time with Pandas: {end_time_pandas - start_time_pandas} seconds")


8
Execution Time with Pandas: 0.00037169456481933594 seconds


In [29]:
#handling missing values
dff["A"]=dff["A"].mask(dff["A"]> 999500, None)
print(dff.tail())
print("")

print(dff.isnull().sum().tail())
print("")

print(dff.fillna(1).tail())

         A  B        C     A_sqrt
999995 NaN  5  1499995   999.9980
999996 NaN  4  1499996   999.9985
999997 NaN  3  1499997   999.9990
999998 NaN  2  1499998   999.9995
999999 NaN  1  1499999  1000.0000

A         500
B           0
C           0
A_sqrt      0
dtype: int64

          A  B        C     A_sqrt
999995  1.0  5  1499995   999.9980
999996  1.0  4  1499996   999.9985
999997  1.0  3  1499997   999.9990
999998  1.0  2  1499998   999.9995
999999  1.0  1  1499999  1000.0000


In [ ]:
#df_concat = dd.concat([df1, df2])
#df = df.dropna()  # Drop rows with NaN values
#df_merged = dd.merge(df1, df2, on='key', how='inner')
#df_sorted = df.sort_values('A').compute()
#df = df.repartition(npartitions=10)
#df = df.repartition(partition_size="100MB")    Repartition by data size

#df.groupby('key').apply(custom_function).compute() #aggregation

#pandas_df = df.compute()  # Convert the entire Dask DataFrame to a pandas DataFrame
#dask_df = dd.from_pandas(pandas_df, npartitions=5) #convert pandas to dask



